Run the code in the first two blocks to get the data, this project uses wikipedia data of cities

Run the code in block three if you stored your OpenAI API key in a `.env` file.

In [ ]:
# wiki_titles = ["Toronto", "Seattle", "San Francisco", "Chicago", "Boston", "Washington, D.C.", "Cambridge, Massachusetts", "Houston"]

In [ ]:
# from pathlib import Path

# import requests
# for title in wiki_titles:
#     response = requests.get(
#         'https://en.wikipedia.org/w/api.php',
#         params={
#             'action': 'query',
#             'format': 'json',
#             'titles': title,
#             'prop': 'extracts',
#             # 'exintro': True,
#             'explaintext': True,
#         }
#     ).json()
#     page = next(iter(response['query']['pages'].values()))
#     wiki_text = page['extract']

#     data_path = Path('data')
#     if not data_path.exists():
#         Path.mkdir(data_path)

#     with open(data_path / f"{title}.txt", 'w') as fp:
#         fp.write(wiki_text)

In [ ]:
# from dotenv import load_dotenv
# import os
# load_dotenv()
# open_api_key = os.getenv("OPENAI_API_KEY")

In [1]:
from llama_index.llms import OpenAI
from llama_index.query_engine import CitationQueryEngine
from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    ServiceContext,
)
from llama_index.vector_stores import MilvusVectorStore

from milvus import default_server

In [2]:
default_server.start()



    __  _________ _   ____  ______
   /  |/  /  _/ /| | / / / / / __/
  / /|_/ // // /_| |/ / /_/ /\ \
 /_/  /_/___/____/___/\____/___/ {Lite}

 Welcome to use Milvus!

 Version:   v2.2.11-lite
 Process:   74039
 Started:   2023-07-21 10:11:36
 Config:    /Users/yujiantang/.milvus.io/milvus-server/2.2.11/configs/milvus.yaml
 Logs:      /Users/yujiantang/.milvus.io/milvus-server/2.2.11/logs

 Ctrl+C to exit ...


In [3]:
vector_store = MilvusVectorStore(
    collection_name="citations",
    host="127.0.0.1",
    port=default_server.listen_port
)

[__internal_register] retry:4, cost: 0.27s, reason: <_InactiveRpcError: StatusCode.UNAVAILABLE, internal: Milvus Proxy is not ready yet. please wait>


In [4]:
service_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0)
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [7]:
documents = SimpleDirectoryReader("./data/").load_data()
index = VectorStoreIndex.from_documents(documents, service_context=service_context, storage_context=storage_context)

In [8]:
query_engine = CitationQueryEngine.from_args(
    index,
    similarity_top_k=3,
    # here we can control how granular citation sources are, the default is 512
    citation_chunk_size=512,
)

In [9]:
response = query_engine.query("Does Seattle or Houston have a bigger airport?")

In [10]:
print(response)

Houston has a bigger airport than Seattle. George Bush Intercontinental Airport (IAH) in Houston is the eighth busiest commercial airport in the United States and the forty-third busiest globally [1]. In 2016, it served 40 million passengers, including 10 million international travelers [1]. On the other hand, Seattle-Tacoma International Airport (Sea-Tac) in Seattle is not mentioned in the provided sources.


In [11]:
for source in response.source_nodes:
    print(source.node.get_text())

Source 1:
Field Joint Reserve Base. The Federal Aviation Administration and the state of Texas selected the Houston Airport System as "Airport of the Year" in 2005, largely due to the implementation of a $3.1 billion airport improvement program for both major airports in Houston.George Bush Intercontinental Airport (IAH), 23 miles (37 km) north of Downtown Houston between Interstates 45 and 69, is the eighth busiest commercial airport in the United States (by total passengers and aircraft movements) and forty-third busiest globally. The five-terminal, five-runway, 11,000-acre (4,500-hectare) airport served 40 million passengers in 2016, including 10 million international travelers. In 2006, the United States Department of Transportation named IAH the fastest-growing of the top ten airports in the United States. The Houston Air Route Traffic Control Center is at Bush Intercontinental.
Houston was the headquarters of Continental Airlines until its 2010 merger with United Airlines with he